In [1]:
import OpenHowNet
hownet_dict = OpenHowNet.HowNetDict()

Initializing OpenHowNet succeeded!


In [4]:
import numpy as np
dataset = ["mams_small\mams_test.raw", "mams_small\mams_train.raw", "mams_small\mams_val.raw",
"mams\mams_test.raw", "mams\mams_train.raw","mams\mams_val.raw","semeval14\\restaurant_test.raw",
"semeval14\\restaurant_train.raw","semeval15\\restaurant_train.raw", "semeval15\\restaurant_test.raw",
            "semeval16\\restaurant_test.raw", "semeval16\\restaurant_train.raw","Z_data\\train.raw", "Z_data\\test.raw", "Z_data\\dev.raw"]
dataset2 = ["semeval14\\laptop_test.raw", "semeval14\\laptop_train.raw"]
dataset3 = ["T_data\\train.raw", "T_data\\test.raw"]
def get_sememe(filename):
    fin = open(filename, 'r', encoding='utf-8', newline='\n', errors='ignore')
    lines = fin.readlines()
    all_sememe = []
    for i in range(0, len(lines), 3):
        left_and_right = lines[i].strip().split("$T$")
        sentence = left_and_right[0] +lines[i+1].strip() + left_and_right[1]
        sememe = []
        token = sentence.split()
        for tok in token:
            tok = tok.lower()
            tok_sememes = hownet_dict.get_sememes_by_word(word=tok, display='list', merge=True, expanded_layer=-1, K=None)
            tok_sememes = [str(s).split("|")[0] for s in tok_sememes]
            sememe.append(tok_sememes)
        all_sememe.append(sememe)

    sememe_data = np.array(all_sememe, dtype="object")
    np.save(filename+".sememe.npy", sememe_data)
root = r"C:\Users\ASUS\OneDrive\桌面\indo-dotGCN\datasets"
for file in dataset3:
    path = root + "\\" + file
    print(path)
    get_sememe(path)

C:\Users\ASUS\OneDrive\桌面\indo-dotGCN\datasets\T_data\train.raw
C:\Users\ASUS\OneDrive\桌面\indo-dotGCN\datasets\T_data\test.raw


In [5]:
from nltk import Tree
import stanza
nlp = stanza.Pipeline('en')

2023-01-10 20:35:59 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2023-01-10 20:35:59 INFO: Use device: gpu
2023-01-10 20:35:59 INFO: Loading: tokenize
2023-01-10 20:35:59 INFO: Loading: pos
2023-01-10 20:35:59 INFO: Loading: lemma
2023-01-10 20:35:59 INFO: Loading: depparse
2023-01-10 20:36:00 INFO: Loading: sentiment
2023-01-10 20:36:01 INFO: Loading: constituency
2023-01-10 20:36:01 INFO: Loading: ner
2023-01-10 20:36:02 INFO: Done loading processors!


In [6]:
import os
import pickle
import numpy as np
datasets = ["mams_small\mams_test.raw", "mams_small\mams_train.raw", "mams_small\mams_val.raw",
"mams\mams_test.raw", "mams\mams_train.raw","mams\mams_val.raw","semeval14\\restaurant_test.raw",
"semeval14\\restaurant_train.raw","semeval15\\restaurant_train.raw", "semeval15\\restaurant_test.raw",
            "semeval16\\restaurant_test.raw", "semeval16\\restaurant_train.raw",]
datasets2 = [
            "Z_data\\train.raw", "Z_data\\test.raw", "Z_data\\dev.raw"]
dataset3 = ["T_data\\train.raw", "T_data\\test.raw"]
fname = r"C:\Users\ASUS\OneDrive\桌面\indo-dotGCN\datasets\semeval14\laptop_test.raw"
def get_const(filename):
    fin = open(filename, 'r', encoding='utf-8', newline='\n', errors='ignore')
    lines = fin.readlines()
    all_tag_path = []
    for i in range(0, len(lines), 3):
        left_and_right = lines[i].strip().split("$T$")
        sentence = left_and_right[0] +lines[i+1].strip() + left_and_right[1]
        sentence = sentence.replace("(", "（")
        sentence = sentence.replace(")", "）")
        try:
            doc = nlp(sentence)
            sen = doc.sentences[0]
            t = Tree.fromstring(str(sen.constituency))  
            leaves_pos = t.treepositions('leaves')
            sentence_tag_path = []
            for leaves in leaves_pos:
                word_tag_path = [t[leaves]]
                for i in range(1, len(leaves)):
                    word_tag_path.append(t[leaves[:-i]].label())
                word_tag_path.reverse()
                sentence_tag_path.append(word_tag_path)

            text = sentence.split()
            # 句法切词和空格切词可能不一致
            path_dic = {}
            flag_dic = {}
            flag_count_dic = {}
            flag_count_use_dic = {}
            # 句子原本词的个数
            
            for w in text:
                value = flag_dic.get(w, "None")
                if value == "None":
                    flag_dic[w] = 1
                    flag_count_dic[w] = 1
                    flag_count_use_dic[w] = 1
                else:
                    flag_dic[w] += 1
            for path in sentence_tag_path:
                value = path_dic.get(path[-1], "None")
                value_n = flag_dic.get(path[-1], 0)
                if value_n == 1:
                    path_dic[path[-1]] = path
                elif value_n > 1:
                    path_dic[path[-1]+str(flag_count_dic[path[-1]])] = path
                    flag_count_dic[path[-1]] += 1

            complete_path = []
            for w in text:
                if flag_dic[w] == 1:
                    complete_path.append(path_dic.get(w, []))
                else:
                    complete_path.append(path_dic.get(w+str(flag_count_use_dic[w]), []))
                    flag_count_use_dic[w] += 1
            all_tag_path.append(complete_path)
        except:
            text = sentence.split()
            all_tag_path.append([[0] for i in range(len(text))])

    data = np.array(all_tag_path, dtype=object)
    cons_path = filename + ".const.npy"
    # print(all_tag_path)
    np.save(cons_path, data)
root = r"C:\Users\ASUS\OneDrive\桌面\indo-dotGCN\datasets"
for dataset in dataset3:
    path = root + "\\" + dataset
    get_const(path)